In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import tensorflow as tf

In [2]:
# Import Data
# mstats_df = pd.read_csv('AMAZON AWS URL')
mstats_df = pd.read_csv('cleaned_dnd_monsters.csv')
mstats_df.head(10)

,name,url,cr,type,size,ac,hp,legendary,str,dex,con,int,wis,cha,fly,swim
0,aarakocra,https://www.aidedd.org/dnd/monstres.php?vo=aar...,0.25,humanoid (aarakocra),Medium,12,13,NaN,10.0,14.0,10.0,11.0,12.0,11.0,Yes,No
1,aboleth,https://www.aidedd.org/dnd/monstres.php?vo=abo...,10.00,aberration,Large,17,135,Legendary,21.0,9.0,15.0,18.0,15.0,18.0,No,No
2,acolyte,https://www.aidedd.org/dnd/monstres.php?vo=aco...,0.25,humanoid (any race),Medium,10,9,NaN,10.0,10.0,10.0,10.0,14.0,11.0,No,No
3,adult-black-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,14.00,dragon,Huge,19,195,Legendary,23.0,14.0,21.0,14.0,13.0,17.0,Yes,Yes
4,adult-blue-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,16.00,dragon,Huge,19,225,Legendary,25.0,10.0,23.0,16.0,15.0,19.0,Yes,No
5,adult-brass-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,13.00,dragon,Huge,18,172,Legendary,23.0,10.0,21.0,14.0,13.0,17.0,Yes,No
6,adult-bronze-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,15.00,dragon,Huge,19,212,Legendary,25.0,10.0,23.0,16.0,15.0,19.0,Yes,Yes
7,adult-copper-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,14.00,dragon,Huge,18,184,Legendary,23.0,12.0,21.0,18.0,15.0,17.0,Yes,No
8,adult-gold-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,17.00,dragon,Huge,19,256,Legendary,27.0,14.0,25.0,16.0,15.0,24.0,Yes,Yes
9,adult-green-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,15.00,dragon,Huge,19,207,Legendary,23.0,12.0,21.0,18.0,15.0,17.0,Yes,Yes


In [3]:
# Remove Name & URL
mstats_df = mstats_df.drop(['name','url'], axis=1)
# Generate our categorical variable lists
mstats_cat = mstats_df.dtypes[mstats_df.dtypes == "object"].index.tolist()
# Check the number of unique values in each column
mstats_df[mstats_cat].nunique()

type         47
size          6
legendary     1
fly           2
swim          2
dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(mstats_df[mstats_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(mstats_cat)
encode_df.head()

g:\Anaconda\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,type_aberration,type_beast,type_celestial,type_construct,type_dragon,type_elemental,type_fey,type_fiend,type_fiend (demon),"type_fiend (demon, orc)",...,size_Large,size_Medium,size_Small,size_Tiny,legendary_Legendary,legendary_nan,fly_No,fly_Yes,swim_No,swim_Yes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
mstats_df = mstats_df.merge(encode_df,left_index=True, right_index=True)
mstats_df = mstats_df.drop(mstats_cat,1)
mstats_df.columns

g:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Index(['cr', 'ac', 'hp', 'str', 'dex', 'con', 'int', 'wis', 'cha',
       'type_aberration', 'type_beast', 'type_celestial', 'type_construct',
       'type_dragon', 'type_elemental', 'type_fey', 'type_fiend',
       'type_fiend (demon)', 'type_fiend (demon, orc)',
       'type_fiend (demon, shapechanger)', 'type_fiend (devil)',
       'type_fiend (devil, shapechanger)', 'type_fiend (shapechanger)',
       'type_fiend (yugoloth)', 'type_giant', 'type_giant (stone giant)',
       'type_humanoid (aarakocra)', 'type_humanoid (any race)',
       'type_humanoid (bullywug)', 'type_humanoid (dwarf)',
       'type_humanoid (elf)', 'type_humanoid (gnoll)', 'type_humanoid (gnome)',
       'type_humanoid (goblinoid)', 'type_humanoid (grimlock)',
       'type_humanoid (human)', 'type_humanoid (human, shapechanger)',
       'type_humanoid (kenku)', 'type_humanoid (kobold)',
       'type_humanoid (kuo-toa)', 'type_humanoid (lizardfolk)',
       'type_humanoid (merfolk)', 'type_humanoid (orc)',
      

In [6]:
# Split our preprocessed data into our features and target arrays
y = mstats_df["cr"].values
X = mstats_df.drop(['cr'],axis=1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Preprocess numerical data for neural network
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
mstats_df.columns

Index(['cr', 'ac', 'hp', 'str', 'dex', 'con', 'int', 'wis', 'cha',
       'type_aberration', 'type_beast', 'type_celestial', 'type_construct',
       'type_dragon', 'type_elemental', 'type_fey', 'type_fiend',
       'type_fiend (demon)', 'type_fiend (demon, orc)',
       'type_fiend (demon, shapechanger)', 'type_fiend (devil)',
       'type_fiend (devil, shapechanger)', 'type_fiend (shapechanger)',
       'type_fiend (yugoloth)', 'type_giant', 'type_giant (stone giant)',
       'type_humanoid (aarakocra)', 'type_humanoid (any race)',
       'type_humanoid (bullywug)', 'type_humanoid (dwarf)',
       'type_humanoid (elf)', 'type_humanoid (gnoll)', 'type_humanoid (gnome)',
       'type_humanoid (goblinoid)', 'type_humanoid (grimlock)',
       'type_humanoid (human)', 'type_humanoid (human, shapechanger)',
       'type_humanoid (kenku)', 'type_humanoid (kobold)',
       'type_humanoid (kuo-toa)', 'type_humanoid (lizardfolk)',
       'type_humanoid (merfolk)', 'type_humanoid (orc)',
      

In [26]:
# Define the deep learning model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=40, activation="relu", input_dim=67))
nn_model.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
10/10 [==============================] - 1s 3ms/step - loss: 0.6492 - accuracy: 0.1000
Epoch 2/50
10/10 [==============================] - 0s 3ms/step - loss: -2.7765 - accuracy: 0.0733
Epoch 3/50
10/10 [==============================] - 0s 3ms/step - loss: -6.5325 - accuracy: 0.0667
Epoch 4/50
10/10 [==============================] - 0s 3ms/step - loss: -10.5608 - accuracy: 0.0700
Epoch 5/50
10/10 [==============================] - 0s 3ms/step - loss: -15.6151 - accuracy: 0.0700
Epoch 6/50
10/10 [==============================] - 0s 3ms/step - loss: -22.0431 - accuracy: 0.0700
Epoch 7/50
10/10 [==============================] - 0s 3ms/step - loss: -30.5757 - accuracy: 0.0700
Epoch 8/50
10/10 [==============================] - 0s 2ms/step - loss: -42.0800 - accuracy: 0.0700
Epoch 9/50
10/10 [==============================] - 0s 4ms/step - loss: -56.8236 - accuracy: 0.0700
Epoch 10/50
10/10 [==============================] - 0s 5ms/step - loss: -75.4716 - accuracy: 0.0700
Epo